In [1]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached langchain-0.3.14-py3-none-any.whl (1.0 MB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl (50 kB)
  Using cached langchain_community-0.3.14-py3-none-any.whl (2.5 MB)
  Using cached langchain_text_splitters-0.3.4-py3-none-any.whl (27 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached langchain_chroma-0.1.4-py3-none-any.whl (10 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-win_amd64.whl (442 kB)
  Using cached langsmith-0.2.10-py3-none-any.whl (326 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl (411 kB)
  Using cached SQLAlchemy-2.0.36-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl (431 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached openai-1.59.3-py3-none-any.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.29.2 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 5.29.2 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./docs/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [1]:
%pip install --upgrade --quite \
    langchain-pinecone \
    langchain-openai \
    langchain

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\Scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: --quite


In [3]:
# ChromaDB -> Pinecone 변경

import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)
retrieved_docs

[Document(id='b0bc8cef-7744-4110-a8ac-d421beb9db9c', metadata={'source': './docs/tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 

In [ ]:
%pip install -U langchain langchainhub --quiet

In [8]:
from langchain import hub
from langchain_openai import ChatOpenAI

# hub에서 rag용 prompt 사
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI()

from langchain.chains import RetrievalQA

retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=retriever,
  chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain({"query": query})

print(ai_message)


c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '5천만원인 직장인의 소득세는 근로소득에 해당하며, 해당 과세기간 동안 받는 근로소득의 총합액에서 근로소득공제를 적용한 금액으로 계산됩니다. 이에 대한 세액은 대통령령에 따라 정해집니다.'}


# table 변환 후 로드

In [10]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 마크다운으로 테이블을 생성하면 성능향상을 기대할 수 있음
loader = Docx2txtLoader('./docs/tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [11]:
# ChromaDB -> Pinecone 변경

import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-table-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [12]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# hub에서 rag용 prompt 사
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI()

retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=retriever,
  chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain({"query": query})

print(ai_message)

c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '5천만원인 경우, 연봉에 대한 소득세율은 약 5.15%이며, 소득세는 약 257,500원입니다.'}


# Markdown 형태로 변환 후 테스트

In [14]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 마크다운으로 테이블을 생성하면 성능향상을 기대할 수 있음
loader = Docx2txtLoader('./docs/tax_with_markdown_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [15]:

import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-table-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [16]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# hub에서 rag용 prompt 사
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI()

retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=retriever,
  chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain({"query": query})

print(ai_message)

c:\Users\kimdo\AppData\Local\pypoetry\Cache\virtualenvs\rag-llm어플리케이션실습-dgGHJXyp-py3.10\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '5천만원인 직장인의 소득세는 연간 약 679,000원입니다. 연봉 5천만원은 2021년 기준 20%의 소득세율이 적용됩니다. '}


In [18]:
query

'연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [20]:
# 가장 적합한 문서가 낮은 우선순위로 선택된 것을 확인
# 질문 수정이 필요
retriever = database.as_retriever(search_kwargs={"k":10})
retriever.invoke(query)

[Document(id='8f27e252-97e4-4237-89be-ba89a9a184e1', metadata={'source': './docs/tax_with_markdown_table.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선

# 키워드 사전을 활용한 효율 개선

In [22]:
query = '연봉 5천만원인 거주자a의 소득세는 얼마인가요?'
retriever = database.as_retriever(search_kwargs={"k":10})
retriever.invoke(query)

[Document(id='513bb499-48e8-4fd7-9240-723bc107148a', metadata={'source': './docs/tax_with_markdown_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준 | 세율 | |---------------------------|-------------------------------------------| | 1,400만원 이하 | 과세표준의 6% | | 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) | | 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) | | 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) | | 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) | | 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40%) | | 5억원 초과 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42%) | | 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45%) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 

In [23]:
# 문서에 거주자의 소득세로 나와있기 때문
# 벡터 상 거주자와 직장인의 유사도가 낮아 이러한 문제가 발생

ai_message = qa_chain({"query": query})

print(ai_message)

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?', 'result': '5천만원인 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액의 15%)로 계산됩니다. 연봉이 5천만원인 경우 세율에 따라 적용되는 종합소득산출세액을 확인하여 소득세를 계산할 수 있습니다.세율에 따라 연봉 5천만원에 해당하는 종합소득산출세액을 계산하면 84만원 + (1,400만원을 초과하는 금액의 15%)가 됩니다.'}


In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()


In [31]:
new_question = dictionary_chain.invoke({"question": "연봉 5천만원인 직장인의 소득세는 얼마인가요?"})
print(new_question)

new_question = dictionary_chain.invoke({"question": "연봉 5천만원인 거주자의 소득세는 얼마인가요?"})
print(new_question)


변경된 질문 : 연봉 5천만원을 받는 거주자의 소득세는 얼마인가요?
변경하지 않습니다.


In [32]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [33]:
ai_response = tax_chain.invoke({"question": query})


In [34]:
ai_response

{'query': '변경된 질문 : 연봉 5천만원인 사람의 소득세는 얼마인가요?',
 'result': '연봉이 5천만원인 사람의 종합소득에 대한 세율은 624만원 + (5,000만원을 초과하는 금액의 24%)입니다. 종합소득산출세액을 계산하여 해당하는 세액으로 적용됩니다. 5천만원을 초과하는 금액의 24%가 소득세로 부과됩니다.'}